# Obtaining DIA sources from DP1

Example tutorial on DIA sources: [link](https://dp1.lsst.io/tutorials/notebook/201/notebook-201-5.html)

Example tutorial on ECDFS: [link](https://dp1.lsst.io/tutorials/notebook/301/notebook-301-4.html)

Schema: [link](https://sdm-schemas.lsst.io/)

In [ ]:
from lsst.rsp import get_tap_service
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
service = get_tap_service("tap")
assert service is not None

## ECDFS

ECDFS center.

In [ ]:
ra_cen, dec_cen = 53.16, -28.10

In [ ]:
query = """SELECT diaSourceId, band, visit,
        midpointMjdTai, ra, dec, psfFlux, psfFluxErr, isDipole,
        apFlux, apFluxErr, scienceFlux, scienceFluxErr
        FROM dp1.DiaSource
        WHERE CONTAINS(POINT('ICRS', ra, dec),
        CIRCLE('ICRS', {}, {}, 1.0)) = 1
        ORDER BY diaSourceId ASC""".format(ra_cen, dec_cen)
print(query)
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'

In [ ]:
results = job.fetch_result().to_table()
print(len(results))

Select high SNR ones on the difference image.

In [ ]:
sel = np.abs(results['psfFlux'] / results['psfFluxErr']) > 5
sel &= np.abs(results['apFlux'] / results['apFluxErr']) > 5
#sel &= (results['band'] == 'r')
print(np.sum(sel))

Non-dipoles.

In [ ]:
sel2 = ~results['isDipole']
print(np.sum(sel2))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), layout='constrained')
ax1.plot(results['ra'][sel], results['dec'][sel],
         'o', ms=2, mew=0, alpha=0.1, color='grey')
ax1.set_xlabel('RA [deg]')
ax1.set_ylabel('DEC [deg]')
ax1.invert_xaxis()
    
ax2.plot(results['psfFlux'][sel], results['psfFluxErr'][sel],
         'o', ms=2, mew=0, alpha=0.1, color='grey')
ax2.set_xlabel('PSF Flux')
ax2.set_ylabel('PSF Flux Error')
ax2.set_xscale('log')
ax2.set_yscale('log')

In [ ]:
bins = 50 

plt.figure()
_ = plt.hist(np.arcsinh(results['apFlux'] - results['psfFlux']),
             bins=bins, histtype='step', density=True)
_ = plt.hist(np.arcsinh(results['apFlux'][sel] - results['psfFlux'][sel]),
             bins=bins, histtype='step', density=True)
_ = plt.hist(np.arcsinh(results['apFlux'][sel & sel2] - results['psfFlux'][sel & sel2]),
             bins=bins, histtype='step', density=True)

_ = plt.axvline(0, ls=':', c='k')

Check the long tail on the right?

In [ ]:
bins = 50 

plt.figure()
_ = plt.hist(np.arcsinh(np.abs(results['apFlux'] - results['psfFlux'])),
             bins=bins, histtype='step', density=True)
_ = plt.hist(np.arcsinh(np.abs(results['apFlux'][sel] - results['psfFlux'][sel])),
             bins=bins, histtype='step', density=True)
_ = plt.hist(np.arcsinh(np.abs(results['apFlux'][sel & sel2] - results['psfFlux'][sel & sel2])),
             bins=bins, histtype='step', density=True)

_ = plt.axvline(0, ls=':', c='k')

In [ ]:
bins = np.linspace(-2, 2, 50)

plt.figure()
_ = plt.hist(np.log10(np.abs(results['apFlux'] / results['psfFlux'])),
             bins=bins, histtype='step', density=True)
_ = plt.hist(np.log10(np.abs(results['apFlux'][sel] / results['psfFlux'][sel])),
             bins=bins, histtype='step', density=True)
_ = plt.hist(np.log10(np.abs(results['apFlux'][sel & sel2] / results['psfFlux'][sel & sel2])),
             bins=bins, histtype='step', density=True)

_ = plt.axvline(0, ls=':', c='k')